# Creating a YOLO model

We first install the YOLO library

In [0]:
!git clone https://github.com/joheras/darknet-colab

In [0]:
cd darknet-colab/

In [0]:
!make

In [0]:
!pip install matplotlib
!pip install clodsa
!pip install imutils
!pip install shutil

Now, we download the necessary files. The following files are employed for fine-tuning the network and configuring the dataset.

In [0]:
!wget https://www.dropbox.com/s/0js92hbmkozgn46/funciones.py?dl=1 -O funciones.py
!wget https://www.dropbox.com/s/k8tj95uc25x2u7d/pascal2yolo_1class.py?dl=1 -O pascal2yolo_1class.py
!wget https://pjreddie.com/media/files/darknet53.conv.74 -O darknet53.conv.74
!wget https://www.dropbox.com/s/5pkkrx82q9dd7ml/detect.py?dl=1 -O detect.py
!wget https://www.dropbox.com/s/s5ogv96lxqxhr4v/generateXML.py?dl=1 -O generateXML.py

In [0]:
from funciones import *
from pascal2yolo_1class import *
from detect import *

We download the dataset. If you are working with Google Colab, you have several options to download the dataset in this notebook, see the available options in the [LabelDetection documentation](https://github.com/ancasag/LabelDetection).

In [0]:
!cp /content/drive/My\ Drive/datasets/dataset.zip dataset.zip
!unzip dataset.zip

We start the training process.

In [0]:
!./darknet detector train dataset/classes.data dataset/train.cfg darknet53.conv.74 -dont_show

Finally, we evaluate the model to know its performance in an independent test set. 

In [0]:
!./darknet detector map dataset/classes.data dataset/test.cfg backup/train_final.weights -thresh 0.25

At the end you will have your model in the file pesos. Those weights can be included in the application to be employed with new images. 


## Data distillation

After training a model with the annotated images, it is possible to apply a data distillation procedure to create a model using the unlabelled images. You can only apply this techique if there were unlabelled images in your dataset. 



We first install the library for ensemble methods.

In [0]:
!git clone https://github.com/ancasag/ensembleObjectDetection.git

In [0]:
cd ensembleObjectDetection/TestTimeAugmentation

In [0]:
!pip install mrcnn
!pip install keras-retinanet
!pip install gluoncv
!pip install mxnet

In [0]:
import testTimeAugmentation
import function
import os
import shutil
import argparse
import ensembleOptions
from mainTTA import tta
from imutils import paths

In [0]:
pathImg = '/content/darknet-colab/dataset/unlabelled/'

In [0]:
yolo = testTimeAugmentation.DarknetYoloPred('/content/darknet-colab/backup/train_final.weights', '/content/darknet-colab/dataset/classes.names','/content/darknet-colab/dataset/train.cfg')

In [0]:
myTechniques = [ "histo","hflip","none"]

In [0]:
option = "affirmative"

In [0]:
tta(yolo,myTechniques,pathImg,option)

We convert the annotations of the new predictions to txt

In [0]:
cd /content/darknet-colab/

In [0]:
principal('/content/darknet-colab/dataset/unlabelled') 

We move the jpg and txt to the train carpet

In [0]:
moverTxt('/content/darknet-colab/dataset/unlabelled/', '/content/darknet-colab/dataset/train/labels/')
moverImg('/content/darknet-colab/dataset/unlabelled/', '/content/darknet-colab/dataset/train/JPEGImages/')

In [0]:
!./darknet detector train dataset/classes.data dataset/train.cfg darknet53.conv.74 -dont_show

In [0]:
!./darknet detector map dataset/classes.data dataset/test.cfg backup/train_final.weights -thresh 0.25

---------------------------


# Using the model in LabelDetection

If you want to use the trained model with LabelDetection, you must download the following files:

- /content/darknet-colab/backup/train_final.weights
- /content/darknet-colab/dataset/train.cfg
- /content/darknet-colab/dataset/classes.names